In [14]:
#importing libraries
import pandas as pd

In [15]:
#importing the time frame we had set 
df_accurate_timeframe = pd.read_csv("/Users/Anne/LE_CODING_🐥/DSF/projekt/accurate_timeframe.csv")

# Closed stores

In [16]:
#importing the excel csv with the dates of when the stores are all closed
df_closing_holidays = pd.read_excel("/Users/Anne/LE_CODING_🐥/DSF/projekt/feiertage ladenschliessung.xlsx")

In [17]:
#setting the datetime column of the accurate time frame to datetime 
df_accurate_timeframe["datetime"]= pd.to_datetime(df_accurate_timeframe["datetime"])

In [18]:
# setting the Datum column to datetime and renaming it to datetime, so it'll match the accurate time frame
df_closing_holidays["datetime"]= pd.to_datetime(df_closing_holidays["Datum"])

In [19]:
#making sure it's of type datetime
df_closing_holidays.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Feiertag  18 non-null     object        
 1   Datum     18 non-null     datetime64[ns]
 2   datetime  18 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(1)
memory usage: 560.0+ bytes


In [20]:
# function that resamples the daily time series data to an hourly

hourly_standard = pd.DataFrame()

for _, row in df_closing_holidays.iterrows():
    date_range = pd.date_range(row['Datum'], periods=24, freq='H')
    temp_df = pd.DataFrame({'Feiertag': [row['Feiertag']] * 24, 'Datum': date_range})
    hourly_standard = pd.concat([hourly_standard, temp_df])

# Reset the index of the new DataFrame
hourly_standard.reset_index(drop=True, inplace=True)


In [21]:
# assigning a 1.0 to the dates where the stores are closed
hourly_standard["stores_closed_on_that_day"] = 1.0

In [22]:
#dropping the column Feiertage
hourly_closed_holidays = hourly_standard.drop("Feiertag", axis = 1)

In [23]:
#renaming the Datum column to datetime
hourly_closed_holidays.rename(columns = {"Datum": "datetime"}, inplace = True)

In [24]:
#making sure that there are no duplicates 
hourly_closed_holidays.duplicated().sum()

0

In [25]:
#merging the df containing the dates of the closed stores with the accurate time frame
hourly_closed_holidays_final = pd.merge(df_accurate_timeframe, hourly_closed_holidays, how ="left", on="datetime")

In [26]:
#filling all the dates where the stores are open with 0.0
hourly_closed_holidays_final = hourly_closed_holidays_final.fillna(0.0)

In [27]:
# saving the dt to a csv
hourly_closed_holidays_final.to_csv('/Users/Anne/LE_CODING_🐥/DSF/projekt/accurate_timeframe_+_stores_closed.csv', index=False)

# Big events

In [28]:
#importing the excel csv with the dates of big events
df_grossevents = pd.read_excel("/Users/Anne/LE_CODING_🐥/DSF/projekt/grossevents zürich.xlsx")

In [29]:
# setting the Datum column to datetime and renaming it to datetime, so it'll match the accurate time frame
df_grossevents["datetime"]= pd.to_datetime(df_grossevents["Datum"])

In [30]:
# function that resamples the daily time series data to an hourly
hourly_standard_g_e = pd.DataFrame()

for _, row in df_grossevents.iterrows():
    date_range = pd.date_range(row['datetime'], periods=24, freq='H')
    temp_df = pd.DataFrame({'Event': [row['Event']] * 24, 'datetime': date_range})
    hourly_standard_g_e = pd.concat([hourly_standard_g_e, temp_df])

# Reset the index of the new DataFrame
hourly_standard_g_e.reset_index(drop=True, inplace=True)

In [31]:
# assigning a 1.0 to the dates when big events are happening
hourly_standard_g_e["Event"] = 1.0

In [32]:
#merging the df containing the dates of the big events with the accurate time frame
hourly_gross_events_final = pd.merge(df_accurate_timeframe, hourly_standard_g_e, how ="outer", on="datetime")

In [33]:
#filling all the dates where no big events are happening with 0.0
hourly_gross_events_final = hourly_gross_events_final.fillna(0.0)

In [34]:
# saving the dt to a csv
hourly_gross_events_final.to_csv('/Users/Anne/LE_CODING_🐥/DSF/projekt/gross_events_acurate_time_frame.csv', index=False)
